In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")
%cd drive/MyDrive/ssd_pytorch

In [ ]:
!git clone https://github.com/dorukuzucu/ssd_object_detection

In [ ]:
%cd ssd_object_detection/
import os
from pathlib import Path
from models.ssd import SSD
from src.model.trainer import Trainer
from models.metrics import MultiboxLoss
from torch.utils.data import DataLoader
from src.data.dataset import MarketDataset

In [ ]:
# define paths
%cd ..
project_path = os.getcwd()
dataset_path = os.path.join(project_path, "data")
train_dataset_path = os.path.join(dataset_path, "train")

project_path,dataset_path,train_dataset_path

In [ ]:
# define datasets
from ssd_object_detection.src.data.utils import collate_fn

train_dataset = MarketDataset(root_dir=train_dataset_path,train=True, in_memory=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=32,collate_fn=collate_fn)
val_dataset = MarketDataset(root_dir=train_dataset_path,train=False, in_memory=True)
val_loader = DataLoader(dataset=train_dataset,batch_size=32,collate_fn=collate_fn)


In [ ]:
# define model and loss function
import torch
ssd_model = SSD(300,5)
criterion = MultiboxLoss(num_cls=5,priors=ssd_model.priors)
ssd_model.load_state_dict(torch.load(os.path.join(project_path, "results")+os.sep+"model25.pth"))

In [ ]:
# define configurations
conf = {
    "number_of_epochs": 100,
    "val_every": 5,
    "optimizer": "SGD",
    "lr": 0.05,
    "device": "gpu",
    "result_path": os.path.join(project_path,"results"),
    "batch_size": 32
}
conf

In [ ]:
# start training
trainer = Trainer(model=ssd_model, criteria=criterion,train_loader=train_loader,val_loader=val_loader,config=conf)
trainer.train_supervised()